# Example GUI for Freping and AFC



In [1]:
# optionally make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Pull in all the necessary python modules

from osp_control import OspControl
import numpy as np
import ipywidgets as widgets
import threading
import time

In [3]:
# create a connection to a local OSP process
osp = OspControl()

# or connect to a PCD device
# osp = OspControl("ospboard.lan") 
# osp = OspControl("192.168.86.38") 

In [4]:
# set the number of bands and mute mic
osp.send({"method": "set", "data": {"num_bands": 11}})
osp.send_chan({"alpha": 1});

In [5]:
# get the current number of bands
vals = osp.send({"method": "get"})
cur_bands = vals["num_bands"]

In [6]:
# Create Controls

freping = widgets.Checkbox(
    description='Freping',
)

# create 11 sliders. We will only display the ones necessary
freping_alpha = []
for band in range(11):
    freping_alpha.append(widgets.FloatSlider(
        value=0,
        min=-.5,
        max=.5,
        step=0.01,
        description=f"Band {band}",
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.2f'
    ))

afc = widgets.Checkbox(
    description='AFC',
)

afc_type = widgets.Dropdown(
    options=[('FXLMS', 1), ('IPNLMS', 2), ('SLMS', 3)],
    value=1,
    description='Type:',
)

play_w = widgets.Dropdown(
    options=[('diner', 'Freping/MPEG_es01_s.wav'), ('animals', 'Freping/MPEG_es03_s.wav'), ('sound', 'Freping/MPEG_te19.wav')],
    value='Freping/MPEG_es01_s.wav',
    description='Play:',
)

play_b = widgets.ToggleButton(
    description='Play',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Play File',
    icon='play'
)

In [7]:
# called when controls change
def update_osp(k=None):
    w = k['owner']
    if w == freping:
        osp.send_chan({"freping": w.value})
        return
    if w == afc:
        osp.send_chan({"afc": w.value})
        return
    if w == afc_type:
        osp.send_chan({"afc_type": w.value})
        return
    fvec = [freping_alpha[x].value for x in range(vals['num_bands'])]
    osp.send_chan({"freping_alpha": fvec})

In [8]:
# when play button is clicked
def play_cb(k):
    w = k['owner']
    osp.send_chan({"audio_filename": play_w.value, "audio_play": 1, "audio_alpha": 1}, 'left')

In [9]:
# set callbacks
play_b.observe(play_cb, names='value')
freping.observe(update_osp, names='value')
afc.observe(update_osp, names='value')
afc_type.observe(update_osp, names='value')
for band in range(11):
    freping_alpha[band].observe(update_osp, names='value')

In [10]:
# build really simple GUI
play_box = widgets.HBox([play_w, play_b], layout={'border': 'solid 2px'})
afc_box = widgets.HBox([afc, afc_type], layout={'border': 'solid 2px'})
fsliders = widgets.VBox([freping_alpha[x] for x in range(cur_bands)])
left_box = widgets.VBox([freping, fsliders], layout={'border': 'solid 2px'})
gui = widgets.VBox([left_box, afc_box, play_box])
gui